# Shapley effects for the Rust Model

In [1]:
# Import statements as in simulation_convergence.ipynb.
import matplotlib.pyplot as plt
#import yaml
import numpy as np
from numpy.testing import assert_allclose
from ruspy.simulation.simulation import simulate
from ruspy.model_code.fix_point_alg import calc_fixp
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.estimation.estimation_transitions import create_transition_matrix

# My imports.
from ruspy.estimation.estimation import estimate
from ruspy.model_code.demand_function import get_demand 
from python.econsa_shapley import get_shapley
from python.econsa_shapley import _r_condmvn

### Get Shapley effects for the Rust model
 1. Calculate variance-covariance matrix and mean vector for input variables with and without $\theta_{32}$.
 2. Define functions `x_all()`, `x_cond()` and `rust_model()`, and function arguments.
 3. Run `get_shapley()`.

In [2]:
# Set simulating variables.
disc_fac = 0.9999
num_buses = 50
num_periods = 120
gridsize = 1000
# We use the cost parameters and transition probabilities from the replication.
params = np.array([10.07780762, 2.29417622])
trans_probs = np.array([0.39189182, 0.59529371, 0.01281447])
scale = 1e-3

init_dict_simulation = {
    "simulation": {
        "discount_factor": disc_fac,
        "periods": num_periods,
        "seed": 123,
        "buses": num_buses,
    },
    "plot": {"gridsize": gridsize},
}

In [3]:
# Calucalte objects necessary for the simulation process. See documentation for details.
num_states = 200
costs = calc_obs_costs(num_states, lin_cost, params, scale)

trans_mat = create_transition_matrix(num_states, trans_probs)
ev = calc_fixp(trans_mat, costs, disc_fac)[0]

In [4]:
# Use one init_dict for get_demand() and estimate().
init_dict_estimation = {
    'model_specifications': {
        'discount_factor': disc_fac,
        'number_states': num_states,
        'maint_cost_func': 'linear',
        'cost_scale': 1e-3
    },
    'optimizer': {
        'approach': 'NFXP',
        'algorithm': 'scipy_L-BFGS-B',
        'gradient': 'Yes'
    },
    
}

In [5]:
# Function for simulating variance-covariance matrices and mean vectors.
def get_cov_and_mean(num_sim,
                     ev, 
                     costs, 
                     trans_mat, 
                     init_dict_simulation, 
                     init_dict_estimation
                    ):
    '''
    Calculate variance-covariance matrix (cov) and mean vector (mean) of simulated data.
    
    Arguments
    ----------------
    num_sim: int
        Number of simulations for the data from which cov and mean are calculated.
    num_input_variables: int
        Either 4 or 5. Calculate cov and mean for all five or only four variables.
        
    Returns
    ----------------
    cov: nd.array
        Variance-covariance matrix of simulated data with shape (num_sim, num_input_variables).
    mean: n.array
        Mean vector of simulated data with shape (, num_input_variables).
    '''
    
    parameter_estimates_4_inputs = np.zeros((num_sim, 4))
    parameter_estimates_5_inputs = np.zeros((num_sim, 5))
    
    for i in np.arange(num_sim):
        
        init_dict_simulation['simulation']['seed'] = +i
        
        df = simulate(init_dict_simulation["simulation"], ev, costs, trans_mat)
        data = df[['state', 'decision', 'usage']].copy()
        
        result_transitions_nfxp, result_fixp_nfxp = estimate(init_dict_estimation, data)
            
        # Record only two of three transition probabilities i.o.t. avoid singularity of the covariance matrix.
        parameter_estimates_4_inputs[i, :] = np.concatenate((result_transitions_nfxp['x'][:2], result_fixp_nfxp['x']))
        # All five inputs.
        parameter_estimates_5_inputs[i, :] = np.concatenate((result_transitions_nfxp['x'], result_fixp_nfxp['x']))
        
        assert_allclose(parameter_estimates_5_inputs[i, :2].sum(), 1.0, rtol=0.02)
            
    cov_4_inputs = np.cov(parameter_estimates_4_inputs.T)
    mean_4_inputs = np.mean(parameter_estimates_4_inputs, axis=0)
    
    cov_5_inputs = np.cov(parameter_estimates_5_inputs.T)
    mean_5_inputs = np.mean(parameter_estimates_5_inputs, axis=0)
            
    return cov_4_inputs, mean_4_inputs, cov_5_inputs, mean_5_inputs

#### 1. Calculate variance-covariance matrices and mean vectors
Due to singularity issues of the covariance matrix if all transition probabilities are simulated in the same simulation process, we need to simulate the two transition probabilities first ($\theta_{30}$, $\theta_{31}$) and calculate the third one from the other two.

In [6]:
# Define number of simulations.
num_sim = 100

In [7]:
%%time

cov_4_inputs, mean_4_inputs, cov_5_inputs, mean_5_inputs = get_cov_and_mean(num_sim, 
                                                                            ev, 
                                                                            costs, 
                                                                            trans_mat, 
                                                                            init_dict_simulation, 
                                                                            init_dict_estimation
                                                                           )

Wall time: 5min 16s


In [8]:
cov_4_inputs

array([[ 3.83524018e-05, -3.84560831e-05,  3.96808147e-04,
         1.03823854e-04],
       [-3.84560831e-05,  4.11072503e-05, -1.54448662e-04,
        -1.20611625e-05],
       [ 3.96808147e-04, -1.54448662e-04,  2.20057498e+00,
         9.20980218e-01],
       [ 1.03823854e-04, -1.20611625e-05,  9.20980218e-01,
         4.30902457e-01]])

In [9]:
mean_4_inputs

array([ 0.39149667,  0.59529333, 10.64765122,  2.51636884])

In [10]:
mean_5_inputs

array([ 0.39149667,  0.59529333,  0.01321   , 10.64765122,  2.51636884])

#### 2. Define functions `x_all()`, `x_cond()` and `rust_model()`, and function arguments.
The idea is to include all five input variables, i.e. $\theta_{32}$ as well.
 - `x_all()` will need another specification working with the 4x4 covariance matrix only ($\theta_{32}$ is calculated by $\theta_{30}$ and $\theta_{31}$).
 - `x_cond()` will need to change. If none or all indices of the transition probabilities are included in sj or sjc, one need to perform sampling differently. In all other cases sampling will work with a 5x5 covariance matrix. 
 - `rust_model()` will be simplified since it now takes five inputs. No need of calculating $\theta_{32}$ within the function as this is done in the sampling functions.

In [ ]:
def x_all(n):
    samples = np.zeros((5, n))
    parameter_samples = cp.MvNormal(mean_4, cov_4).sample(n)
    samples[:2, :] = parameter_samples[:2, :]
    samples[2, :] = 1 - np.sum(parameter_samples[:2, :])  # better sample from dist. via cov_int.take as below.
    samples[3:, :] = parameter_samples[2:, :]
    return samples

In [ ]:
def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        
        if all(x in subset_j for x in [0, 1, 2]) == True:
            
            subset_j_new = subset_j.copy()
            
            # Drop value 2 from subset_j.
            subset_j_new = subset_j_new[subset_j_new != 2]
            
            cov_int = np.array(cov_5_inputs)
            cov_int = cov_int.take(subset_j_new, axis = 1)
            cov_int = cov_int[subset_j_new]
            distribution = cp.MvNormal(mean[subset_j_new], cov_int)
            
            # Sample of shape (len(subset_j_new), n).
            auxiliary_sample = distribution.sample(n)
            
            # Initialize array for storing values for the returned sample.
            out_sample = np.zeros((len(subset_j), n))
            
            # Get index where subset_j == 2.
            index_theta_32 = np.where(subset_j==2)[0][0]
            
            # Check whether index where subset_j == 2 is first index in subset_j.
            if index_theta_32 > 0:
                out_sample[:index_theta_32, :] = auxiliary_sample[:index_theta_32, :]
            else:
                pass
            #parameter_sample[np.where(subset_j==2), :] = 1 - np.sum(
            #    (auxiliary_sample[np.where(subset_j==1), :], auxiliary_sample[np.where(subset_j==0), :])
            #    )
            
            # Sample values for theta_32. Cannot do this like this if correlation 
            # coefficients w.r.t. theta_32 are not close to zero.
            cov_int_theta_32 = np.array(cov_5_inputs).take(2, axis = 1)[2]
            
            distribution_theta_32 = cp.MvNormal(mean[2], cov_int_theta_32)
            
            out_sample[index_theta_32, :] = distribution_theta_32.sample(n)
            
            # Check whether index where subset_j == 2 is the last index in subset_j.
            if index_theta_32 < (len(subset_j) - 1):
                out_sample[(index_theta_32 + 1):, :] = auxiliary_sample[np.where(subset_j_new==2)[0][0]:, :]
            else:
                pass
            
            return parameter_sample
            
        else:
            
            cov_int = np.array(cov)
            cov_int = cov_int.take(subset_j, axis = 1)
            cov_int = cov_int[subset_j]
            distribution = cp.MvNormal(mean[subset_j], cov_int)
            
            return distribution.sample(n)
    
    else:
        
        if all(x in subsetj_conditional for x in [0, 1, 2]) == True:
            
            
            
        else:
            return _r_condmvn(
                n, mean = mean_5_inputs, cov = cov_5_inputs, dependent_ind = subset_j, given_ind = subsetj_conditional, x_given = xjc
            )

In [23]:
array_ones = np.ones(5)

In [43]:
array_ones[:(np.where(subset_j==4)[0][0])]

array([1., 1., 1., 1.])

In [47]:
len(subset_j_new)-1

4

In [45]:
np.where(subset_j[-1])

(array([0], dtype=int64),)

In [16]:
subset_j = np.array([2, 0, 3, 1, 4])
subset_j_new = subset_j.copy()

In [58]:
subset_j_new

array([2, 0, 3, 1, 4])

In [17]:
subset_j_new = subset_j_new[subset_j_new != 2]

In [18]:
subset_j_new

array([0, 3, 1, 4])

In [61]:
subset_j_new

array([0, 3, 1, 4])

In [62]:
#[subset_j_new[i] -1 if subset_j_new[i] > 2 else subset_j_new[i] for i in np.arange(4)]

In [63]:
subset_j_new[np.where(subset_j_new==0)] = 1

In [64]:
subset_j_new

array([1, 3, 1, 4])